In [4]:
%pip install --upgrade --quiet  langchain-pinecone==0.1.1 langchain_community langchain langchainhub python-dotenv langchain_core langchain_openai pinecone

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-tests 0.3.20 requires langchain-core<1.0.0,>=0.3.63, but you have langchain-core 0.2.43 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip list

Package                   Version
------------------------- ---------------
aiohappyeyeballs          2.6.1
aiohttp                   3.12.14
aiohttp-retry             2.9.1
aiosignal                 1.4.0
annotated-types           0.7.0
anyio                     4.9.0
appnope                   0.1.4
asttokens                 3.0.0
attrs                     25.3.0
certifi                   2025.7.14
cffi                      1.17.1
charset-normalizer        3.4.2
comm                      0.2.2
dataclasses-json          0.6.7
debugpy                   1.8.15
decorator                 5.2.1
distro                    1.9.0
executing                 2.2.0
frozenlist                1.7.0
h11                       0.16.0
httpcore                  1.0.9
httpx                     0.28.1
httpx-sse                 0.4.1
idna                      3.10
iniconfig                 2.1.0
ipykernel                 6.29.5
ipython                   9.4.0
ipython_pygments_lexers   1.1.1
jedi             

In [ ]:
%pip install pinecone streamlit --upgrade


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import streamlit as st
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain.chains import RetrievalQA
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPrompxtTemplate
from langchain_openai import ChatOpenAI


st.set_page_config(page_title="소득세 챗봇", page_icon="🤖")
st.title("🤖 소득세 챗봇")
st.caption("소득세에 관련된 답변을 모두 답해드립니다! ")

load_dotenv()  # .env 파일에서 환경 변수 로드

## pinecone 활용
# Initialize a Pinecone client with your API key
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

def get_ai_message(user_message):
    # 임베딩 생성하기
    embedding = OpenAIEmbeddings(model="text-embedding-3-large")
    # 데이터를 처음 저장할 때
    index_name = 'tax-markdown-index'
    # llm에 질의 하기
    llm = ChatOpenAI(model="gpt-3.5-turbo")
    # Initialize the PineconeVectorStore
    database = PineconeVectorStore.from_existing_index(
        index_name=index_name,
        embedding=embedding
    )

    
    ## 프롬프트 생성
    prompt = hub.pull("rlm/rag-prompt")

    retriever=database.as_retriever(search_kwargs={'k':4})
    qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=retriever,
        chain_type_kwargs={"prompt": prompt}  # 프롬프트를 지정
    )

    query = "연봉 5000만원의 직장인의 소득세는 얼마인가요? "

    dictionary = "\n".join([
        "사람을 나타내는 표현 -> 거주자"
    ])
    prompt = ChatPromptTemplate.from_template(f"""
                사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요
                만약에 변경할 필요가 없다고 판단이 되면 사용자의 질문을 변경하지 않아도 됩니다
                사전 : {dictionary}
                질문 : {{question}}
                """)

    dictionary_chain = prompt | llm | StrOutputParser()
    tax_chain = {"query": dictionary_chain} | qa_chain
    ai_message = tax_chain.invoke({"question": user_message})

    return ai_message




if 'message_list' not in st.session_state:
    st.session_state.message_list = []
for message in st.session_state.message_list : 
    with st.chat_message(message["role"]):
        st.write(message["content"])

if user_question := st.chat_input(placeholder="소득세에 관련된 궁금한 내용들을 말씀해 주세요"):
    with st.chat_message("user"):
        st.write(user_question)
    st.session_state.message_list.append({"role": "user", "content": user_question})

    ai_message = get_ai_message(user_question)

    with st.chat_message("ai"):
        st.write(ai_message)
    st.session_state.message_list.append({"role": "ai", "content": ai_message})



ModuleNotFoundError: No module named 'streamlit'

In [ ]:
# embedding 생성
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain.chains import RetrievalQA
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import time

load_dotenv()  # .env 파일에서 환경 변수 로드

# 임베딩 생성하기
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

## pinecone 활용
# Initialize a Pinecone client with your API key
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

# 데이터를 처음 저장할 때
index_name = 'tax-index'
# llm에 질의 하기
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")


# Initialize the PineconeVectorStore
database = PineconeVectorStore.from_documents(
    documents=[],  # Start with an empty list
    embedding=embedding,
    index_name=index_name
)


query = "연봉 5000만원의 직장인의 소득세는 얼마인가요? "

## 프롬프트 생성
prompt = hub.pull("rlm/rag-prompt")
retriever=database.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}  # 프롬프트를 지정
)


dictionary = "\n".join([
    "사람을 나타내는 표현 -> 거주자",
    "근로자 -> 직장인",
])
prompt = ChatPromptTemplate.from_template(f"""
            사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요
            만약에 변경할 필요가 없다고 판단이 되면 사용자의 질문을 변경하지 않아도 됩니다
            사전 : {dictionary}
            질문 : {{question}}
            """)

dictionary_chain = prompt | llm | StrOutputParser()

new_question = dictionary_chain.invoke({
    "question": query,
    "dictionary": dictionary  # 여기를 꼭 넣어줘야 함!
})
tax_chain = {"query": dictionary_chain} | qa_chain
ai_response = tax_chain.invoke({"question":query})
ai_response 

